In [16]:
import math
import os 
import pandas as pd 

In [4]:
# EDA

In [2]:
def preprocess_information(file_name: str) -> pd.DataFrame:
    couriers = open(file_name)
    couriers = [i.split("\t") for i in couriers]
    couriers = [[j.strip() for j in i] for i in couriers]
    couriers_df = pd.DataFrame(couriers[1:], columns = couriers[0])
    return couriers_df


In [4]:
instance_parameters = preprocess_information("data/0o50t75s1p100/instance_parameters.txt").astype(int)

In [5]:
meters_per_min, pickup_service_minutes, dropoff_service_minutes, target_click_to_door, maximum_click_to_door, pay_per_order, guaranteed_pay_per_hour = instance_parameters.iloc[0]

In [6]:
instance_parameters

,meters_per_minute,pickup service minutes,dropoff service minutes,target click-to-door,maximum click-to-door,pay per order,guaranteed pay per hour
0,427,4,4,40,90,10,15


In [8]:
couriers = preprocess_information("data/0o50t75s1p100/couriers.txt")

In [9]:
orders = preprocess_information("data/0o50t75s1p100/orders.txt")

In [10]:
restaurants = preprocess_information("data/0o50t75s1p100/restaurants.txt")

In [11]:
restaurants.columns = ['restaurant', 'restaurant_x', 'restaurant_y']

In [12]:
order_restaurant = orders.merge(restaurants, left_on = 'restaurant', right_on = 'restaurant', how = 'inner')

In [13]:
order_restaurant['x'] = order_restaurant['x'].astype(int)
order_restaurant['y'] = order_restaurant['y'].astype(int)
order_restaurant['restaurant_x'] = order_restaurant['restaurant_x'].astype(int)
order_restaurant['restaurant_y'] = order_restaurant['restaurant_y'].astype(int)

In [14]:
def calculate_euclidean_distance(source_x, source_y, destination_x, destination_y):
    return ((source_x - destination_x)**2 + (source_y - destination_y) ** 2) ** 0.5


In [17]:
order_restaurant['euclidean_distance'] = order_restaurant.apply(lambda x: math.dist((x['x'],x['y']),(x['restaurant_x'], x['restaurant_y'])), axis = 1)

In [18]:
def generate_schedule(on_time : int, off_time : int) -> list:
    return [i for i in range(on_time, off_time + 1)]

In [19]:
order_restaurant['time_taken_to_travel'] = order_restaurant['euclidean_distance'].apply(lambda x: x/meters_per_min)

In [20]:
courier_can_pick_order = {} 
courier_current_location = {} 
courier_time_taken = {} 

for _, v in couriers.iterrows():
    courier, current_x, current_y, on_time, off_time = v
    courier_current_location[courier] = (int(current_x), int(current_y))



for _, v in couriers.iterrows():
    courier, current_x, current_y, on_time, off_time = v
    schedule_range = generate_schedule(int(on_time), int(off_time))
    
    for _, x in order_restaurant.iterrows():
        order, destination_x, destination_y, placement_time, restaurant, ready_time, source_x, source_y, euclidean_distance, time_taken_to_travel = x


        if ready_time <= off_time and ready_time >= on_time:
            if courier not in courier_can_pick_order.keys():
                courier_can_pick_order[courier] = [order]
            else:
                courier_can_pick_order[courier].append(order)

In [21]:
all_orders = list(order_restaurant['order'])

courier_time_taken = {} 
courier_order_assigned = {} 
for courier_id, order_list in courier_can_pick_order.items():

    smallest_time_taken = 1e9

    for order_id in order_list:
        order_subset = order_restaurant[order_restaurant['order'] == order_id]
        current_x, current_y = courier_current_location[courier_id]
        order_x, order_y, restaurant_x, restaurant_y = order_subset['x'].iloc[0], order_subset['y'].iloc[0], order_subset['restaurant_x'].iloc[0], order_subset['restaurant_y'].iloc[0]
        
        initial_travel_distance = calculate_euclidean_distance(current_x, current_y, restaurant_x, restaurant_y)
        order_travel_distance = order_subset['euclidean_distance'].iloc[0]

        total_travel_distance = initial_travel_distance + order_travel_distance
        total_time_taken = total_travel_distance/meters_per_min

        if total_time_taken < smallest_time_taken:
            if order_id in all_orders:
                smallest_time_taken = total_time_taken 
                courier_time_taken[courier_id] = smallest_time_taken
                courier_order_assigned[courier_id] = order_id 
                all_orders.remove(order_id)
            else:
                previous_order_id = courier_order_assigned[courier_id]
                all_orders.append(previous_order_id)
                
                smallest_time_taken = total_time_taken 
                courier_time_taken[courier_id] = smallest_time_taken
                courier_order_assigned[courier_id] = order_id 
                all_orders.remove(order_id)


KeyError: 'c16'

In [24]:
len(all_orders)

236

In [23]:
current_x, current_y

(9292, 1183)

In [22]:
courier_current_location

{'c1': (9755, 1693),
 'c2': (3049, 7734),
 'c3': (8343, 752),
 'c4': (5471, 0),
 'c5': (10252, 2493),
 'c6': (7038, 26),
 'c7': (10663, 5087),
 'c8': (8256, 8490),
 'c9': (9212, 1109),
 'c10': (8464, 562),
 'c11': (2922, 1330),
 'c12': (9028, 952),
 'c13': (1661, 4683),
 'c14': (1687, 4999),
 'c15': (9774, 1718),
 'c16': (9292, 1183),
 'c17': (2883, 6253),
 'c18': (8941, 1446),
 'c19': (7488, 6328),
 'c20': (5190, 3023),
 'c21': (9803, 3525),
 'c22': (2096, 2524),
 'c23': (5668, 2563),
 'c24': (2579, 1730),
 'c25': (7223, 8872),
 'c26': (2457, 6197),
 'c27': (7922, 293),
 'c28': (1740, 3600),
 'c29': (2353, 5855),
 'c30': (5376, 2954),
 'c31': (2163, 5416),
 'c32': (5223, 8891),
 'c33': (9434, 6880),
 'c34': (9587, 1490),
 'c35': (3559, 8157),
 'c36': (4697, 194),
 'c37': (3698, 2338),
 'c38': (4304, 1224),
 'c39': (2337, 6855),
 'c40': (7381, 2226),
 'c41': (3053, 7738),
 'c42': (9621, 1530),
 'c43': (5406, 11),
 'c44': (4963, 8135),
 'c45': (9658, 1573),
 'c46': (4971, 109),
 'c47': 

In [143]:
order_restaurant

,order,x,y,placement_time,restaurant,ready_time,restaurant_x,restaurant_y,euclidean_distance,time_taken_to_travel
0,o1,8317,5587,743,r1,753,7818,3668,1982.816683,4.643599
1,o8,7202,4887,612,r1,622,7818,3668,1365.802694,3.198601
2,o21,6589,3457,336,r1,346,7818,3668,1246.981155,2.920331
3,o26,6302,5401,623,r1,633,7818,3668,2302.508415,5.392291
4,o38,10350,4937,594,r1,614,7818,3668,2832.204971,6.632799
...,...,...,...,...,...,...,...,...,...,...
247,o234,10441,2921,92,r90,116,8515,5778,3445.565991,8.069241
248,o239,5088,5063,715,r90,728,8515,5778,3500.793339,8.198579
249,o235,9093,4164,201,r91,202,8694,5518,1411.565443,3.305774
250,o249,7327,7640,610,r92,640,9344,5622,2853.175950,6.681911


In [90]:
# Heuristic - start from the nearest and continue 

In [91]:
class plan_route:
    def __init__(self, order_restaurant, couriers):
        self.couriers = couriers 
        self.order_restaurant = order_restaurant
    
    def check_valid(self, )``

SyntaxError: invalid syntax (4163469444.py, line 6)